## 4.1 模型构造

### 4.1.1 继承Module类来构造模型

Module 类是 nn 模块里提供的一个模型构造类，是所有神经网络模块的基类，可以继承它来定义想要的模型

重载了Module类的 __init__ 函数和 forward 函数

- __init__() 创建模型参数
- forward() 定义前向计算即正向传播
- 无须定义反向传播函数。系统将通过自动求梯度而自动生成反向传播所需的backward函数

In [12]:
import torch
from torch import nn

class MLP(nn.Module):
    # 声明带有模型参数的层，这里声明了两个全连接层
    def __init__(self, **kwargs):
        # 调用MLP父类Module的构造函数来进行必要的初始化。这样在构造b实例时还可以指定其他函数
        # 参数，如“模型参数的访问、初始化和共享”一节将介绍的模型参数params
        super(MLP, self).__init__(**kwargs)
        self.hidden = nn.Linear(784, 256) # 隐藏层
        self.act = nn.ReLU()
        self.output = nn.Linear(256, 10)  # 输出层


    # 定义模型的前向计算，即如何根据输入x计算返回所需要的模型输出
    def forward(self, x):
        a = self.act(self.hidden(x))
        return self.output(a)

In [18]:
X = torch.rand(2, 784)
net = MLP()
print(net)

MLP(
  (hidden): Linear(in_features=784, out_features=256, bias=True)
  (act): ReLU()
  (output): Linear(in_features=256, out_features=10, bias=True)
)


### 4.1.2 Module的子类

#### 4.1.2.1 Sequential类

当模型的前向计算为简单串联各个层的计算时，Sequential 类可以通过更加简单的方式定义模型。它可以接收一个子模块的有序字典（OrderedDict）或者一系列子模块作为参数来逐一添加 Module 的实例，而模型的前向计算就是将这些实例按添加的顺序逐一计算

In [17]:
net = nn.Sequential(
        nn.Linear(784, 256),
        nn.ReLU(),
        nn.Linear(256, 10), 
        )
print(net)

Sequential(
  (0): Linear(in_features=784, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)


#### 4.1.2.2 ModuleList类

ModuleList 接收一个子模块的列表作为输入，然后也可以类似 List 那样进行 append 和 extend 操作

In [19]:
net = nn.ModuleList([nn.Linear(784, 256), nn.ReLU()])
net.append(nn.Linear(256, 10)) # # 类似List的append操作
print(net[-1])  # 类似List的索引访问
print(net)
# net(torch.zeros(1, 784)) # 会报NotImplementedError

Linear(in_features=256, out_features=10, bias=True)
ModuleList(
  (0): Linear(in_features=784, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)


Sequential 和 ModuleList 二者区别

- ModuleList 仅仅是一个储存各种模块的列表，这些模块之间没有联系也没有顺序（所以不用保证相邻层的输入输出维度匹配）
- ModuleList 没有实现 forward 功能, 执行 net(torch.zeros(1, 784)) 会报 NotImplementedError
- Sequential 内的模块需要按照顺序排列，要保证相邻层的输入输出大小相匹配
- 内部 forward 功能已经实现

#### 4.1.2.3 ModuleDict类

ModuleDict接收一个子模块的字典作为输入, 然后也可以类似字典那样进行添加访问操作

In [21]:
net = nn.ModuleDict({
    'linear': nn.Linear(784, 256),
    'act': nn.ReLU(),
})
net['output'] = nn.Linear(256, 10) # 添加
print(net['linear']) # 访问
print(net.output)
print(net)
# net(torch.zeros(1, 784)) # 会报NotImplementedError

Linear(in_features=784, out_features=256, bias=True)
Linear(in_features=256, out_features=10, bias=True)
ModuleDict(
  (linear): Linear(in_features=784, out_features=256, bias=True)
  (act): ReLU()
  (output): Linear(in_features=256, out_features=10, bias=True)
)


和ModuleList一样，ModuleDict实例仅仅是存放了一些模块的字典，并没有定义forward函数需要自己定义

### 4.1.3 构造复杂的模型

In [22]:
class FancyMLP(nn.Module):
    def __init__(self, **kwargs):
        super(FancyMLP, self).__init__(**kwargs)

        self.rand_weight = torch.rand((20, 20), requires_grad=False) # 不可训练参数（常数参数）
        self.linear = nn.Linear(20, 20)

    def forward(self, x):
        x = self.linear(x)
        # 使用创建的常数参数，以及nn.functional中的relu函数和mm函数
        x = nn.functional.relu(torch.mm(x, self.rand_weight.data) + 1)

        # 复用全连接层。等价于两个全连接层共享参数
        x = self.linear(x)
        # 控制流，这里我们需要调用item函数来返回标量进行比较
        while x.norm().item() > 1:
            x /= 2
        if x.norm().item() < 0.8:
            x *= 10
        return x.sum()

### 小结

- 可以通过继承Module类来构造模型。
- Sequential、ModuleList、ModuleDict类都继承自Module类。
- 与Sequential不同，ModuleList和ModuleDict并没有定义一个完整的网络，它们只是将不同的模块存放在一起，需要自己定义forward函数。
- 虽然Sequential等类可以使模型构造更加简单，但直接继承Module类可以极大地拓展模型构造的灵活性。

## 4.2 模型参数的访问、初始化和共享

In [24]:
import torch
from torch import nn
from torch.nn import init

net = nn.Sequential(nn.Linear(4, 3), nn.ReLU(), nn.Linear(3, 1))  # pytorch已进行默认初始化

print(net)
X = torch.rand(2, 4)
Y = net(X).sum()

Sequential(
  (0): Linear(in_features=4, out_features=3, bias=True)
  (1): ReLU()
  (2): Linear(in_features=3, out_features=1, bias=True)
)


### 4.2.1 访问模型参数

通过 Module 类的 parameters() 或者 named_parameters 方法来访问所有参数（以迭代器的形式返回）

In [25]:
print(type(net.named_parameters()))
for name, param in net.named_parameters():
    print(name, param.size())

<class 'generator'>
0.weight torch.Size([3, 4])
0.bias torch.Size([3])
2.weight torch.Size([1, 3])
2.bias torch.Size([1])


通过方括号 [] 来访问网络的任一层。索引 0 表示隐藏层为Sequential实例最先添加的层。

In [26]:
for name, param in net[0].named_parameters():
    print(name, param.size(), type(param))

weight torch.Size([3, 4]) <class 'torch.nn.parameter.Parameter'>
bias torch.Size([3]) <class 'torch.nn.parameter.Parameter'>


Parameter 会自动被添加到模型的参数列表里

In [27]:
class MyModel(nn.Module):
    def __init__(self, **kwargs):
        super(MyModel, self).__init__(**kwargs)
        self.weight1 = nn.Parameter(torch.rand(20, 20))
        self.weight2 = torch.rand(20, 20)
    def forward(self, x):
        pass

n = MyModel()
for name, param in n.named_parameters():
    print(name)

weight1


可以根据data来访问参数数值，用grad来访问参数梯度

In [28]:
weight_0 = list(net[0].parameters())[0]
print(weight_0.data)
print(weight_0.grad) # 反向传播前梯度为None
Y.backward()
print(weight_0.grad)

tensor([[ 0.4605,  0.2187, -0.0052,  0.1480],
        [-0.4731,  0.2223,  0.2270, -0.2448],
        [-0.4647, -0.3081,  0.1854, -0.3373]])
None
tensor([[-0.5121, -0.6148, -0.6578, -0.6878],
        [ 0.0586,  0.1211,  0.1239,  0.1313],
        [ 0.0000,  0.0000,  0.0000,  0.0000]])


### 4.2.2 初始化模型参数

### 4.2.3 自定义初始化方法

### 4.2.4 共享模型参数

## 4.3 模型参数的延后初始化